# 📍 NLP Term Project
⭐ **Team 5**: 2277018 Seoyeon Ye, 2277025 Eunsang Lee, 2277031 Ahyun 

- Setup
  - set parameters
  - load data
- Construct KB
- Categorization using LLM
- Define prompt
- Generate answer
- Evaluate
- Save the answers

## Setup

In [ ]:
!pip3 install -qU python-dotenv langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv faiss-cpu pymupdf4llm llama_index

### set parameters

In [ ]:
api_key = "YOUR_KEY_HERE"
data_path = "./" # folder path containing samples.csv

UPSTAGE_API_KEY = api_key

### load data

In [ ]:
# read csv file
import os
import pandas as pd

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    return prompts, answers

In [ ]:
prompts, answers = read_data(os.path.join(data_path, 'testset.csv'))

In [ ]:
print(len(prompts))
prompts

## Construct KB

In [ ]:
# get documents list
documents = os.listdir('./KB')

for d in documents:
    if '.pdf' not in d:
        documents.remove(d)
        
print(documents)

In [ ]:
# pdf preprocessing
def remove_newlines_except_after_period(text):
    return re.sub(r'(?<!\.)(\n|\r\n)', ' ', text)

In [ ]:
import pymupdf4llm
from langchain_upstage import UpstageEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = UpstageEmbeddings(model="embedding-passage", api_key=UPSTAGE_API_KEY)

# Initialize a dictionary to hold vector stores for each document
vector_stores = {}

for document in documents:
    # 1. Open PDF as Markdown
    chunks = pymupdf4llm.to_markdown(
        './KB/' + document,
        page_chunks=True,
        write_images=False
    )
    
    # Collect splits for the current document
    splits = [remove_newlines_except_after_period(chunk_dict['text']) for chunk_dict in chunks]
    
    # 2. Create embeddings for the current document
    doc_embeddings = embeddings.embed_documents(splits)
    
    # 3. Create Vector Store for the current document
    vector_store = FAISS.from_texts(splits, embedding=embeddings)
    
    # Store the vector store in the dictionary with the document category as the key
    category = document.split('-')[0].lower()
    vector_stores[category] = vector_store

In [ ]:
print(vector_stores.keys())
print(type(vector_stores['business']))

## Categorization using LLM

In [ ]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import PromptTemplate

llm = ChatUpstage(api_key = UPSTAGE_API_KEY, temperature=0)

prompt_template_option = PromptTemplate.from_template(
    """
    # Request
    You are an AI assistant designed to provide accurate and concise answers using the provided context. 
    Please follow these instructions carefully to ensure the best response:
    
    1. Analyze the provided Question and determine the most appropriate category(1~6) from the list below.
    2. If the question is in Korean, the answer must always be (2) regardless of its content.
    3. If the Question is **not in Korean**, choose one category from (1), (3), (4), (5), and (6) based on its content.
    4. Provide the final answer in the following format (no alphabet, only numbers):
       [ANSWER]: (X)  # Replace X with the category number (1~6)
       
    # Categories
    (1) Business 
    (2) Ewha[Specific school regulations]
    (3) History 
    (4) Law 
    (5) Philosophy 
    (6) Psychology
    
    ---
    # Important Notes
    - The answer must only indicate the category number; do not provide explanations or the answer to the question itself.
    - The answer must always be (2) if the question is in Korean.
    - If the question is in a language other than Korean, choose one of categories (1), (3), (4), (5), and (6).
    - Answer which category the question belongs to, not the answer to the question itself. Keep this in mind.
    - Ensure your response strictly matches the required format: `[ANSWER]: (X)`.
    
    ---
    # Question
    {question}
    
    ---
    # Answer
    [ANSWER]: 
    """
)

chain_option = prompt_template_option | llm

## Define prompt

In [ ]:
prompt_template = PromptTemplate.from_template(
    """
    # Request
    You are an AI assistant designed to provide accurate and concise answers using the provided context. 
    Please follow these instructions carefully to ensure the best response:
    
    1. Analyze the context and identify relevant information related to the question.
    2. Explain your reasoning step by step in detail.
    3. Your final answer must be directly based on the context. Do not guess or infer information not present.
    4. Use the following format for your final answer:
       [ANSWER]: (A) `your answer here`
    
    ---
    # Notes
    - Always ensure your response is in English.
    - The answer must be one specific option from the choices provided, based on your analysis of the given context.
    
    ---
    # Question
    {question}
    
    ---
    # Context
    {context}
    """
)

chain = prompt_template | llm

## Generate answer

In [ ]:
# function to extract an answer from response

import re

def extract_option(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((1-6)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again_option(response)

def extract_again_option(response):
    pattern = r"\b[1-6]\b(?!.*\b[1-6]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return '2'
    
def change_category(category):
    if category == "business":
        return '(1)'
    elif category == "ewha":
        return '(2)'
    elif category == "history":
        return '(3)'
    elif category == "law":
        return '(4)'
    elif category == "philosophy":
        return '(5)'
    else: # psychology
        return '(6)'

In [ ]:
responses = []
for prompt in prompts:
    response_option = chain_option.invoke({"question": prompt})
    opt = extract_option(response_option.content)
    
    print(opt)
    
    # business
    if opt == '1':
        # Create the retriever
        vector_store = vector_stores['business']
        retriever = vector_store.as_retriever(
            search_type="mmr",        # Use "similarity" or "mmr" (Maximal Marginal Relevance)
            search_kwargs={"k": 5}    # Retrieve top 5 most similar chunks
        )
        
        relevant_docs = retriever.invoke(prompt)
        relevant_info = "\n".join(doc.page_content for doc in relevant_docs)

        response = chain.invoke({"question": prompt, "context": relevant_info})
        responses.append(response.content)
    
    # ewha    
    elif opt == '2':
        # Create the retriever
        vector_store = vector_stores['ewha.pdf']
        retriever = vector_store.as_retriever(
            search_type="mmr",        # Use "similarity" or "mmr" (Maximal Marginal Relevance)
            search_kwargs={"k": 5}    # Retrieve top 5 most similar chunks
        )
        
        relevant_docs = retriever.invoke(prompt)
        relevant_info = "\n".join(doc.page_content for doc in relevant_docs)

        response = chain.invoke({"question": prompt, "context": relevant_info})
        responses.append(response.content)
    
    # history
    elif opt == '3':
        # Create the retriever
        vector_store = vector_stores['history']
        retriever = vector_store.as_retriever(
            search_type="mmr",        # Use "similarity" or "mmr" (Maximal Marginal Relevance)
            search_kwargs={"k": 5}    # Retrieve top 5 most similar chunks
        )
        
        relevant_docs = retriever.invoke(prompt)
        relevant_info = "\n".join(doc.page_content for doc in relevant_docs)

        response = chain.invoke({"question": prompt, "context": relevant_info})
        responses.append(response.content)
    
    # law
    elif opt == '4':
        # Create the retriever
        vector_store = vector_stores['law']
        retriever = vector_store.as_retriever(
            search_type="mmr",        # Use "similarity" or "mmr" (Maximal Marginal Relevance)
            search_kwargs={"k": 5}    # Retrieve top 5 most similar chunks
        )
        
        relevant_docs = retriever.invoke(prompt)
        relevant_info = "\n".join(doc.page_content for doc in relevant_docs)

        response = chain.invoke({"question": prompt, "context": relevant_info})
        responses.append(response.content)
    
    # philosophy    
    elif opt == '5':
        # Create the retriever
        vector_store = vector_stores['philosophy']
        retriever = vector_store.as_retriever(
            search_type="mmr",        # Use "similarity" or "mmr" (Maximal Marginal Relevance)
            search_kwargs={"k": 5}    # Retrieve top 5 most similar chunks
        )
        
        relevant_docs = retriever.invoke(prompt)
        relevant_info = "\n".join(doc.page_content for doc in relevant_docs)

        response = chain.invoke({"question": prompt, "context": relevant_info})
        responses.append(response.content)
    
    # psychology
    else:
        # Create the retriever
        vector_store = vector_stores['psychology']
        retriever = vector_store.as_retriever(
            search_type="mmr",        # Use "similarity" or "mmr" (Maximal Marginal Relevance)
            search_kwargs={"k": 5}    # Retrieve top 5 most similar chunks
        )
        
        relevant_docs = retriever.invoke(prompt)
        relevant_info = "\n".join(doc.page_content for doc in relevant_docs)

        response = chain.invoke({"question": prompt, "context": relevant_info})
        responses.append(response.content)
    

## Evaluate

In [ ]:
# function to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A-Z)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-Z]\b(?!.*\b[A-Z]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [ ]:
# print accuracy

cnt = 0

for answer, response in zip(answers, responses):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")


    if generated_answer == None:
        continue
    if generated_answer in answer:
        cnt += 1

print()
print(f"acc: {(cnt/len(responses))*100}%")

## Save the answers

In [ ]:
# you can save the answers if you need
"""
file_path = "answers_{0}.txt".format("history")
with open(file_path, "w", encoding='utf-8') as file:
    for item in responses:
        file.write(item + "\n========\n")
"""